# But du notebook
---

Ce notebook sert à récupérer des données en utilisant l'API de genius lyrics afin d'en construire une base de données.

## Import des librairies

In [ ]:
import lyricsgenius

In [ ]:
# Nous récuperons notre clé d'API
with open("genius_client_access_token.txt", "r") as f:
    GENIUS_API_TOKEN = f.read()

## Récupération des données

In [ ]:
# Afin de récupérer nos données, nous devons créer une connexion au serveur genius.
genius = lyricsgenius.Genius(GENIUS_API_TOKEN)

# On peut choisir de marquer les couplets / refrains
genius.remove_section_headers = True 
# On peut choisir d'exclure des chansons basées sur leur titres. Conseillé de ne pas faire (il vaut mieux tout garder et faire le tri après).
genius.excluded_terms = ["(Remix)", "(Live)", "version"] 
genius.timeout = 10
genius.sleep_time = 5
genius.retries = 3

In [ ]:
# On récupère la liste de tous les groupes à récupérer
with open("band_name.txt", "r") as f:
    band_list = f.readlines()

In [ ]:
# récupérer les dernières paroles des groupes

for band in band_list[18:]:
    try:
        artist = genius.search_artist(band.replace("\n", ""))
        # On enregistre les JSON de données au fur et à mesure
        artist.save_lyrics()
    except Exception as e:
       print(e)

In [ ]:
# Il est possible de faire des recherches par artistes
artist = genius.search_artist("Breaking Benjamin")
#artist = genius.search_artist("Breaking Benjamin", max_songs=3, sort="title")
#print(artist.songs)

In [ ]:
# Il est aussi possible de récupérer par chansons
song = artist.song("Diary of Jane")
# or:
# song = genius.search_song("To You", artist.name)
print(song.lyrics)

## Partie 2) Retravail des chansons
---

Dans cette partie, nous partons du principe que les paroles de chansons ont été récupérées et qu'elles ont été déposées dans un dossier appelé "lyrics"

## Import des librairies

In [ ]:
import json
import os
import numpy as np
import pandas as pd

In [ ]:
# chemin relatif du dossier
folder_lyrics = "lyrics" 
# Récupération des fichiers json
file_names = [f for f in os.listdir(folder_lyrics) if f.endswith(".json")]

lyrics_list = []
for json_lyrics_file in file_names:
    file_path = os.path.join(folder_lyrics, json_lyrics_file)

    with open(file_path, "r") as my_file:
        lyrics_list.append(json.load(my_file))

In [ ]:
# Note : du fait du processus de récupération des données, les chansons avec les strings "live", "version" et "remix" ont été exclus. Cela peut mener à des chansons exclus accidentellement
# Par exemple "help I'm alive" de metric. Dans notre cas, on considère que ce n'est pas grave car 1) Il nous reste beaucoup de chansons. 2) Les groupes vont continuer d'en écrire ==> On ne sera jamais à jour
# 3) Le but est de faire une analyse + génération de chansons. Dans les deux cas, on estime qu'une poignée de chansons ne va pas changer la donne.
# Attention : dans les conditions réelles, il est conseillé de stocker toutes les données et de les nettoyer seulement dans un deuxième temps car du coup on ne peut pas évaluer les dégâts.
# Attention bis : on possède aussi des covers (architects a fait une cover de deftones par exemple). On conserve car on suppose qu'un groupe choisit ses covers et qu'elles sont donc dans un univers musical proche du groupe

list_to_exclude = ["acoustic", "session", "instrumental", "(demo)", "(demo", "demo)", "- demo", "radio edit", "single edit", "piano reprise", "unplugged"]
all_songs = []

for k in range(len(lyrics_list)):
    i=0
    band_name = lyrics_list[k]["name"]
    song_list = lyrics_list[k]["songs"]
    for individual_song in song_list:
        song_title = individual_song["title"]
        i+=1
        # On ne récupère pas dans la table de données les morceaux indiquées comme étant des "versions d'une même chanson"
        song_to_exclude = [word in song_title.lower() for word in list_to_exclude]
        if True not in song_to_exclude:
            # On ne garde que les chansons en anglais
            if individual_song["language"] == "en":
                release_date = individual_song["release_date"]
                release_date_for_display = individual_song["release_date_for_display"]
                if "pageviews" in individual_song["stats"]:
                    lyrics_view = individual_song["stats"]["pageviews"]
                else:
                    lyrics_view = np.NaN
                feat = individual_song["featured_artists"]
                primary_artist_name = individual_song["primary_artist"]["name"]
                is_feat = (len(feat) > 0)
                if individual_song["album"] is not None:
                    album_name = individual_song["album"]["name"]
                else:
                    album_name = None
                if len(individual_song["lyrics"].split("Lyrics")) > 1:
                    lyrics = individual_song["lyrics"].split("Lyrics")[1]
                else:
                    lyrics = None
                all_songs.append({"band_name" : band_name, 
                                "song_title" : song_title, 
                                "album_name" : album_name, 
                                "lyrics" : lyrics, 
                                "release_date" : release_date, 
                                "release_date_for_display" : release_date_for_display, 
                                "lyrics_view" : lyrics_view, 
                                "feat" : feat, 
                                "is_feat" : is_feat, 
                                "primary_artist_name" : primary_artist_name})

In [ ]:
# Enregistrement des paroles récupérées
pd.DataFrame(all_songs).to_csv("all_songs.csv", sep=";", index=False, encoding="utf-8")